In [48]:
import torch
from torch import nn
from d2l import torch as d2l

In [49]:
def cov2d(X, K):
    h, w = K.shape
    Y = torch.zeros([X.shape[0] - h + 1, X.shape[1] - w + 1])
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i][j] = ((X[i: i+h, j: j+w]) * K).sum()
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 2.0], [2.0, 3.0]])
cov2d(X, K)

tensor([[20., 27.],
        [41., 48.]])

In [50]:
class myCovLayer(nn.Module):
    def __init__(self, K):
        super().__init__()
        self.kernel = K
        self.bias = torch.zeros(1)

    def forward(self, X):
        return cov2d(X, self.kernel) + self.bias

myCov = myCovLayer(K)
myCov(X)

tensor([[20., 27.],
        [41., 48.]])

In [51]:
X = torch.ones(6, 8)
X[:, 2:5] = 0
X

tensor([[1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.]])

In [52]:
horizontal_kernel = torch.tensor([[1, -1], [1, -1]])
horizontal_cov_Layer = myCovLayer(horizontal_kernel)
horizontal_cov_Layer(X)

tensor([[ 0.,  2.,  0.,  0., -2.,  0.,  0.],
        [ 0.,  2.,  0.,  0., -2.,  0.,  0.],
        [ 0.,  2.,  0.,  0., -2.,  0.,  0.],
        [ 0.,  2.,  0.,  0., -2.,  0.,  0.],
        [ 0.,  2.,  0.,  0., -2.,  0.,  0.]])

In [53]:
K = torch.tensor([[1.0, -1.0]])

In [54]:
Y = cov2d(X, K)
X, Y

(tensor([[1., 1., 0., 0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 0., 1., 1., 1.],
         [1., 1., 0., 0., 0., 1., 1., 1.]]),
 tensor([[ 0.,  1.,  0.,  0., -1.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
         [ 0.,  1.,  0.,  0., -1.,  0.,  0.]]))

### Train a conv layer

In [70]:
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 2e-2  # seems 3e-2 will not convege!!!!

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    print(conv2d.weight)
    print(lr * conv2d.weight.grad)
    print(conv2d.weight.data - lr * conv2d.weight.grad)
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

Parameter containing:
tensor([[[[-0.3310, -0.4066]]]], requires_grad=True)
tensor([[[[-0.8505, -0.3886]]]])
tensor([[[[ 0.5195, -0.0180]]]])
Parameter containing:
tensor([[[[ 0.5195, -0.0180]]]], requires_grad=True)
tensor([[[[0.2458, 0.5968]]]])
tensor([[[[ 0.2737, -0.6148]]]])
epoch 2, loss 11.699
Parameter containing:
tensor([[[[ 0.2737, -0.6148]]]], requires_grad=True)
tensor([[[[-0.4199, -0.1531]]]])
tensor([[[[ 0.6936, -0.4617]]]])
Parameter containing:
tensor([[[[ 0.6936, -0.4617]]]], requires_grad=True)
tensor([[[[0.0934, 0.2962]]]])
tensor([[[[ 0.6001, -0.7578]]]])
epoch 4, loss 3.270
Parameter containing:
tensor([[[[ 0.6001, -0.7578]]]], requires_grad=True)
tensor([[[[-0.2095, -0.0554]]]])
tensor([[[[ 0.8097, -0.7024]]]])
Parameter containing:
tensor([[[[ 0.8097, -0.7024]]]], requires_grad=True)
tensor([[[[0.0315, 0.1486]]]])
tensor([[[[ 0.7781, -0.8510]]]])
epoch 6, loss 0.956
Parameter containing:
tensor([[[[ 0.7781, -0.8510]]]], requires_grad=True)
tensor([[[[-0.1058, -0.0

In [71]:
conv2d.weight

Parameter containing:
tensor([[[[ 0.9303, -0.9459]]]], requires_grad=True)